In [153]:
import pandas as pd
import numpy as np
import sklearn
from collections import Counter
from itertools import dropwhile
from sklearn.cluster import KMeans
import csv


print("Import Complete")

Import Complete


In [154]:
def feature_list(file):
    vocab = []
    with open(file, 'rb') as f:
        reader = csv.reader(f)
        for row in reader:
            word = row[0]
            if word not in vocab:
                vocab.append(word)
            else:
                print("Repeat: {}".format(word))
    return vocab

def label_features(df, features_master):
        for i,row in df.iterrows():
            #message = pruning_dict.remove_nonalphanumeric(row.text)
            message = str(row.question_class)
            features = Counter(message.split()) & features_master
            features = features + features_master
            features = list(np.array(list(features.values())) - 1)
            df.set_value(i,'features',features)
        return df

def create_feature_dataframe(df, features_master):
    return pd.DataFrame(list(df.features), columns=range(len(features_master)))

def cluster_filter2(df, df2, N, features_master, v=False):
    #print("Start loop")
    gpercentages = []
    singles = 0
    clusterer = KMeans(n_clusters=N)
    clusterer.fit(df2)
    transform = clusterer.transform(df2)
    d_center = []
    cluster = []
    for x in transform:
        d_center.append(min(x)**2)
        cluster.append(np.argmin(x))
    df['cluster'] = cluster
    df['d_from_center'] = d_center
    d_center = np.array(d_center)
    mean = np.mean(d_center)
    std = np.std(d_center)
    '''
    for cgroup in range(N):
        group = df.groupby('cluster').get_group(cgroup)
        if group.question_class.count() <= len(df) * .01:
            df = df.drop(group.index)
            singles += 1
    print("# of singles: {}".format(singles))
    print("df length: {}".format(len(df)))
    if singles >= 6 and N <= (len(df) - singles):
        df = cluster_filter2(df, create_feature_dataframe(df, features_master), N, features_master)
        return df
    '''
    for cgroup in range(N):
        group = df.groupby('cluster').get_group(cgroup)
        gpercent = (compute_gpercentage(group, mean, std))
        gpercentages.append(gpercent)
        '''
        if v == True:
            print("Found {} messages of the same form with a gpercent of {}.".format(len(group), gpercent))
            for message in group.question_class.head(3):
                if group.question_class.count() > 1:
                    print(message)
                    print("")
            print("")
        '''
            
    
    median = np.median(np.array(gpercentages))
    if v == True:
        for cgroup in range(N):
            group = df.groupby('cluster').get_group(cgroup)
            gpercent = (compute_gpercentage(group, mean, std))
            if gpercent >= median:
                print_clusters(group, gpercent)
    else:
        pass
        #print(median)
        #print(np.mean(np.array(gpercentages)))
    return df, median

def compute_gpercentage(group, mean, std):

    gscore = 0.0
    for i, row in group.iterrows():
        z = (row.d_from_center - mean) / std
        if z < -0.68:
            gscore += 1
    glength = len(group)
    gpercent = gscore/glength
    return gpercent

def print_clusters(group, gpercent):
    print("Found {} messages of the same form with a gpercent of {}.".format(len(group), gpercent))
    for message in group.question_class.head(5):
        if group.question_class.count() > 1:
            print(message)
            print("")
    print("")
            
    
    

In [155]:
csvdata = "./data/subcategory_2_think_hr_v2.csv"
df = pd.read_csv(csvdata)
df = df[['question_class']]

vocab = feature_list('./HotWords.csv')


features_master = Counter(vocab)
df["features"] = [[0] * len(vocab)] * len(df)
df = label_features(df, features_master)
df2 = create_feature_dataframe(df, features_master)

medians = []
for N in range(25,50):
    print("N: {}".format(N))
    df, median = cluster_filter2(df, df2, N, features_master)
    medians.append([median, N])

best_clustering_index = np.argmax(medians, axis=0)
best_clustering = medians[best_clustering_index[0]][1]
print("The best clustering is {}:".format(best_clustering))
df, median = cluster_filter2(df, df2, best_clustering, features_master, v=True)



Repeat: medicare
Repeat: laws


C:\Users\kerri\Anaconda2\envs\thinkhr\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead


N: 25
N: 26
N: 27
N: 28
N: 29
N: 30
N: 31
N: 32
N: 33
N: 34
N: 35
N: 36
N: 37
N: 38
N: 39
N: 40
N: 41
N: 42
N: 43
N: 44
N: 45
N: 46
N: 47
N: 48
N: 49
The best clustering is 30:
Found 855 messages of the same form with a gpercent of 0.26783625731.
state illinois state mandated lunch break required hourly employees

asking behalf archer credit union please reply jenipher cornelius specifically policy exempt staff requires take pto gone hour exempt long time never seen policy restrictive exempt employees going one step exempt employees coach various sports leave work early employees required take pto even leave early noon taking even one step one employees understand gets paid coach getting paid credit union paid school league coaching paid coaching part get hurt way liability credit union technically clock would responsible workers comp claim

several nurse practitioners work hour shifts urgent care paid salary recently changed paid hourly required take minute lunch break clock sure rule

Found 513 messages of the same form with a gpercent of 0.335282651072.
looking sample whistleblower policy small person ria registered investment advisor firm

pto holiday pay people like use pto extend holiday problem people abusing create policy prohibit employees using pto day holiday

trying assess existing pto policy compliant new cook county paid sick leave ordinance

hello ceo requested write policy says personal dating relationship practicum students work outside work allowed write policy prevents someone relationship outside work please tell write thanks

thank let clarify question wondering legal us write policy says employee harris house allowed date student harris house sort personal relationship student work outside work know answer question thanks


Found 307 messages of the same form with a gpercent of 0.28664495114.
offices san jose parts northern ca sick time requirements area thanks

mmurphy prairieknights com subject national medical support notice good morning quest